In [26]:
import pandas as pd
import wikipedia as w
from bs4 import BeautifulSoup
import requests
import re
import csv 
def retrieve_page(game):
    # try grabbing the page, but if we can't just remove it
    try:
        print(game)
        return w.page(game).content
    except:
        print(f"Game {game} was not found")
        return "Page Not Found"


In [27]:
# scraping nintendo switch games table
best_selling_nintendo = requests.get("https://en.wikipedia.org/wiki/List_of_best-selling_Nintendo_Switch_video_games")
nintendo_soup = BeautifulSoup(best_selling_nintendo.text, 'html.parser')
nintendo_table = nintendo_soup.find_all('table')
switch_games = nintendo_table[1]
# grabs headers which are written around "th" tags
headers = [header.text.strip() for header in switch_games.find_all('th')][:8]



In [28]:
data_rows = switch_games.find_all('tr')
rows = []

# Modified from: 
for row in data_rows:
    value = row.find_all(re.compile('td|th'))
    beautified_value = [ele.text.strip() for ele in value]
    # Remove data arrays that are empty
    if len(beautified_value) == 0:
        continue
    # deal with numbers that are missing entries
    if len(beautified_value) < len(headers):
        beautified_value = [None] + beautified_value
    rows.append(beautified_value)



with open('nintendo_bestsellers_notext.csv', 'w', newline="") as output:
    writer = csv.writer(output)
    writer.writerow(headers)
    writer.writerows(rows)


#drop the repeated first row
ns_bestsellers = pd.read_csv("nintendo_bestsellers_notext.csv")[1:]


#try to grab each page
game_text = ns_bestsellers["Title"].apply(lambda x: retrieve_page(x))



Mario Kart 8 Deluxe
Animal Crossing: New Horizons
Super Smash Bros. Ultimate
The Legend of Zelda: Breath of the Wild
Pokémon Sword and Shield
Super Mario Odyssey
Super Mario Party
Game Super Mario Party was not found
Pokémon Brilliant Diamond and Shining Pearl
Ring Fit Adventure
Pokémon: Let's Go, Pikachu! and Let's Go, Eevee!
Pokémon Legends: Arceus
New Super Mario Bros. U Deluxe
Splatoon 2
Luigi's Mansion 3
Pokémon Scarlet and Violet
Super Mario 3D World + Bowser's Fury
Super Mario 3D All-Stars
Game Super Mario 3D All-Stars was not found
Mario Party Superstars
Splatoon 3
Super Mario Maker 2
Monster Hunter Rise
Game Monster Hunter Rise was not found
Nintendo Switch Sports
The Legend of Zelda: Link's Awakening
Kirby and the Forgotten Land
Mario Tennis Aces
Clubhouse Games: 51 Worldwide Classics
Donkey Kong Country: Tropical Freeze
Hyrule Warriors: Age of Calamity
Kirby Star Allies
The Legend of Zelda: Skyward Sword HD
Fire Emblem: Three Houses
1-2-Switch
Momotaro Dentetsu: Showa, Heise

In [29]:
ns_bestsellers["wiki_page"] = game_text

# drop any pages that are blank

ns_bestsellers.to_csv("nintendo_bestsellers.csv")

In [30]:
!ls

data-cleaning.ipynb             nintendo_bestsellers_notext.csv
nintendo_bestsellers.csv        wiki-scraping.ipynb
